<a href="https://colab.research.google.com/github/bhavanabalraj/Federated-Learning/blob/master/FLWorkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

     |████████████████████████████████| 378kB 4.9MB/s 
     |████████████████████████████████| 1.8MB 12.0MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 2.1MB 41.5MB/s 
     |█████████████████████▉          | 514.8MB 1.4MB/s eta 0:02:56

In [0]:
import torch as tc
from torchvision import datasets, transforms
import syft as sy 

hook = sy.TorchHook(tc)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])])

trainset = datasets.FashionMNIST('./F_MNIST_data', download=True, train=True, transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
trainloader = sy.FederatedDataLoader(trainset.federate((bob,alice)), batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('./F_MNIST_data', download=True, train=False, transform=transform)
testloader = tc.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


In [0]:
from torch import nn, optim
import torch.nn.functional as F

In [0]:
class Classifier(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.output = nn.Linear(64, 10)

    self.dropout = nn.Dropout(p=0.2)

  def forward(self,x):

    x = x.view(x.shape[0],-1)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = F.log_softmax(self.output(x), dim=1)

    return x

In [0]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [1]:
epochs = 5
steps = 0

training_loss, testing_loss = [],[]
for e in range(epochs):
  
  running_loss = 0
  
  for images, labels in trainloader:

    model.send(images.location)

    optimizer.zero_grad()

    log_ps = model(images)
    loss = criterion(log_ps, labels)
    loss.backward()
    optimizer.step()

    model.get()

    running_loss += loss.get()

  else:

    test_loss = 0
    accuracy = 0

    with tc.no_grad():
      
      model.eval()

      for images, labels in testloader:
        log_ps = model(images)
        test_loss = criterion(log_ps, labels)
        test_loss += test_loss.item()

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equality = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor))

    model.train()
    
    training_loss.append(running_loss/len(trainloader))
    testing_loss.append(test_loss/len(testloader))

    print("Epochs :{}/{}".format(e+1, epochs),
    "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
    "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
    "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

NameError: ignored

In [0]:
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/3bd7dea850e936d8cb44adda8200e4e2b5d627e3/intro-to-pytorch/helper.py
import helper

dataiter = iter(testloader)
images,labels = dataiter.next()
image = images[0]

img = image.view(1,784)

model.eval()

with torch.no_grad():
  output = model(img)

ps = torch.exp(output)
model.train()

helper.view_classify(img, ps, version='Fashion')